In [1]:
import re
import time
import requests
from scrapy.http import TextResponse

def getProductName(asinList):
    returnList = []
    errorList = []
    headers = {
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'en-US,en;q=0.9,ko;q=0.8',
        'user-agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Whale/1.5.75.9 Safari/537.36',
    }
    
    
    
    start = time.time()
    for asin in asinList:
        URL = "https://www.amazon.com/product-reviews/{}".format(asin)
        resp = requests.get(URL, headers = headers)

        response = TextResponse(resp.url, body=resp.text, encoding="utf-8")
        link = response.css('body link[rel="canonical"]::attr(href)').get()
        try:
            productName = re.sub(r"(https:\/\/.*\.com\/)|(product-reviews\/)|(\?.*)","",link).split("/")[0]
            returnList.append((asin, productName))
            if(len(returnList)%5==0):
                print("{}개".format(len(returnList)))
        except:
            errorList.append(asin)
    print("time: ", time.time() - start)
    
    
    
    print("에러: {}".format(errorList))
    return returnList


In [2]:
# resultList = getProductName([ "inputList" ])